# 03g: ML Best Practices และ Conclusion

## วัตถุประสงค์การเรียนรู้
- เรียนรู้ Best practices ในการทำ ML projects
- เข้าใจ Model deployment considerations
- รู้จัก Ethical considerations ใน ML
- วางแผนการเรียนรู้ต่อไป

---

## 1. ML Project Best Practices

### Data Quality Checklist
- ✅ ตรวจสอบ missing values
- ✅ หา outliers และ anomalies
- ✅ ตรวจสอบ data leakage
- ✅ validate business logic

### Model Development
- ✅ เริ่มต้นด้วย simple baseline
- ✅ ใช้ cross-validation
- ✅ track experiments
- ✅ ป้องกัน overfitting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# สร้างข้อมูลตัวอย่าง
X, y = make_classification(n_samples=1000, n_features=10, 
                          n_informative=5, random_state=42)

# 1. Learning Curves - ตรวจสอบ overfitting
def plot_learning_curve(model, X, y, title="Learning Curve"):
    train_sizes, train_scores, val_scores = learning_curve(
        model, X, y, cv=5, n_jobs=-1, 
        train_sizes=np.linspace(0.1, 1.0, 10),
        random_state=42
    )
    
    train_mean = train_scores.mean(axis=1)
    train_std = train_scores.std(axis=1)
    val_mean = val_scores.mean(axis=1)
    val_std = val_scores.std(axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training Score')
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
    
    plt.plot(train_sizes, val_mean, 'o-', color='red', label='Cross-Validation Score')
    plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
    
    plt.xlabel('Training Set Size')
    plt.ylabel('Accuracy Score')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return train_mean, val_mean

# ทดสอบกับ Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
train_scores, val_scores = plot_learning_curve(rf_model, X, y, "Random Forest Learning Curve")

# วิเคราะห์ผล
if train_scores[-1] - val_scores[-1] > 0.05:
    print("⚠️  Potential Overfitting detected!")
    print(f"   Training Score: {train_scores[-1]:.3f}")
    print(f"   Validation Score: {val_scores[-1]:.3f}")
    print(f"   Gap: {train_scores[-1] - val_scores[-1]:.3f}")
else:
    print("✅ Model looks good - no significant overfitting")
    print(f"   Training Score: {train_scores[-1]:.3f}")
    print(f"   Validation Score: {val_scores[-1]:.3f}")

## 2. Model Deployment Considerations

### Performance Requirements
- **Latency**: เวลาตอบสนอง
- **Throughput**: จำนวน requests ต่อวินาที
- **Memory Usage**: การใช้หน่วยความจำ
- **Model Size**: ขนาดไฟล์ model

### Monitoring
- **Model Drift**: การเปลี่ยนแปลงของข้อมูล
- **Performance Metrics**: ติดตาม accuracy
- **Data Quality**: ตรวจสอบข้อมูลใหม่
- **Business Metrics**: ผลกระทบต่อธุรกิจ

In [ ]:
# ตัวอย่าง: Model Monitoring
import time
from sklearn.model_selection import train_test_split

class ModelMonitor:
    def __init__(self, model):
        self.model = model
        self.predictions_log = []
        self.performance_log = []
        
    def predict_with_monitoring(self, X, y_true=None):
        """
        ทำนายพร้อมบันทึก monitoring data
        """
        start_time = time.time()
        predictions = self.model.predict(X)
        prediction_time = time.time() - start_time
        
        # บันทึก predictions
        self.predictions_log.extend(predictions)
        
        # คำนวณ performance ถ้ามี ground truth
        if y_true is not None:
            accuracy = accuracy_score(y_true, predictions)
            self.performance_log.append({
                'timestamp': time.time(),
                'accuracy': accuracy,
                'prediction_time': prediction_time,
                'n_samples': len(X)
            })
            
            print(f"Prediction completed:")
            print(f"  Accuracy: {accuracy:.3f}")
            print(f"  Time: {prediction_time:.4f} seconds")
            print(f"  Samples: {len(X)}")
            print(f"  Avg time per sample: {prediction_time/len(X)*1000:.2f} ms")
        
        return predictions
    
    def get_performance_summary(self):
        if not self.performance_log:
            return "No performance data available"
            
        df = pd.DataFrame(self.performance_log)
        return df.describe()

# ทดสอบ monitoring
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ฝึก model
rf_model.fit(X_train, y_train)

# สร้าง monitor
monitor = ModelMonitor(rf_model)

# ทดสอบ prediction with monitoring
predictions = monitor.predict_with_monitoring(X_test, y_test)

print("\nPerformance Summary:")
print(monitor.get_performance_summary())

## 3. Ethical Considerations

### Bias และ Fairness
- **Data Bias**: ข้อมูลมี bias หรือไม่
- **Algorithmic Bias**: algorithm เลือกปฏิบัติหรือไม่
- **Representation**: กลุ่มต่างๆ ได้รับการเป็นตัวแทนเท่าเทียมหรือไม่
- **Impact Assessment**: ผลกระทบต่อกลุ่มต่างๆ

### Privacy และ Security
- **Data Privacy**: ข้อมูลส่วนบุคคลได้รับการปกป้อง
- **Model Security**: ป้องกัน adversarial attacks
- **Data Governance**: นโยบายการใช้ข้อมูล
- **Compliance**: ปฏิบัติตามกฎหมาย (GDPR, etc.)

In [ ]:
# ตัวอย่าง: Bias Detection
def check_prediction_bias(model, X, y, sensitive_feature_idx):
    """
    ตรวจสอบ bias ในการทำนายตาม sensitive feature
    """
    predictions = model.predict(X)
    
    # แบ่งกลุ่มตาม sensitive feature (เช่น เพศ, อายุ)
    sensitive_feature = X[:, sensitive_feature_idx]
    threshold = np.median(sensitive_feature)
    
    group_1 = sensitive_feature <= threshold
    group_2 = sensitive_feature > threshold
    
    # คำนวณ accuracy สำหรับแต่ละกลุ่ม
    acc_group_1 = accuracy_score(y[group_1], predictions[group_1])
    acc_group_2 = accuracy_score(y[group_2], predictions[group_2])
    
    # คำนวณ positive rate สำหรับแต่ละกลุ่ม
    pos_rate_group_1 = predictions[group_1].mean()
    pos_rate_group_2 = predictions[group_2].mean()
    
    print("=== Bias Analysis ===")
    print(f"Group 1 (feature <= {threshold:.2f}): {group_1.sum()} samples")
    print(f"  Accuracy: {acc_group_1:.3f}")
    print(f"  Positive Rate: {pos_rate_group_1:.3f}")
    
    print(f"\nGroup 2 (feature > {threshold:.2f}): {group_2.sum()} samples")
    print(f"  Accuracy: {acc_group_2:.3f}")
    print(f"  Positive Rate: {pos_rate_group_2:.3f}")
    
    # คำนวณความแตกต่าง
    acc_diff = abs(acc_group_1 - acc_group_2)
    rate_diff = abs(pos_rate_group_1 - pos_rate_group_2)
    
    print(f"\nDifferences:")
    print(f"  Accuracy Difference: {acc_diff:.3f}")
    print(f"  Positive Rate Difference: {rate_diff:.3f}")
    
    # เตือนถ้ามี bias
    if acc_diff > 0.05 or rate_diff > 0.1:
        print("\n⚠️  Potential bias detected!")
        print("   Consider fairness constraints or bias mitigation techniques")
    else:
        print("\n✅ No significant bias detected")
    
    return acc_diff, rate_diff

# ทดสอบ bias detection
acc_diff, rate_diff = check_prediction_bias(rf_model, X_test, y_test, sensitive_feature_idx=0)

## 4. ML Best Practices Checklist

### Before Starting
- [ ] กำหนดปัญหาและเป้าหมายชัดเจน
- [ ] เข้าใจ business context
- [ ] กำหนด success metrics
- [ ] ตรวจสอบ ethical implications

### Data Phase
- [ ] สำรวจและทำความเข้าใจข้อมูล
- [ ] ตรวจสอบ data quality
- [ ] จัดการ missing values และ outliers
- [ ] แบ่งข้อมูล train/validation/test

### Modeling Phase
- [ ] เริ่มด้วย simple baseline
- [ ] ใช้ cross-validation
- [ ] เปรียบเทียบหลาย algorithms
- [ ] ทำ hyperparameter tuning
- [ ] ตรวจสอบ overfitting

### Evaluation Phase
- [ ] ประเมินด้วย multiple metrics
- [ ] ทดสอบกับ holdout test set
- [ ] วิเคราะห์ errors และ edge cases
- [ ] ตรวจสอบ bias และ fairness

### Deployment Phase
- [ ] วางแผน monitoring strategy
- [ ] เตรียม rollback plan
- [ ] ทดสอบ performance ใน production
- [ ] สร้าง documentation

## 5. Next Steps ในการเรียนรู้ ML

### พื้นฐานที่ควรเสริม
- **Statistics**: Probability, Hypothesis testing
- **Linear Algebra**: Matrix operations, Eigenvalues
- **Calculus**: Derivatives, Optimization
- **Programming**: Python, SQL, Git

### Advanced Topics
- **Deep Learning**: Neural Networks, CNNs, RNNs
- **NLP**: Text processing, Transformers
- **Computer Vision**: Image classification, Object detection
- **Time Series**: Forecasting, Anomaly detection
- **MLOps**: Model deployment, CI/CD for ML

### Practical Experience
- **Kaggle Competitions**: ฝึกฝนกับปัญหาจริง
- **Personal Projects**: สร้าง portfolio
- **Open Source**: contribute ใน ML libraries
- **Internships**: ประสบการณ์ในบริษัท

## สรุปการเรียนรู้ทั้งหมด

### สิ่งที่เราได้เรียนรู้

#### 1. Machine Learning Fundamentals
- ความแตกต่างระหว่าง Descriptive และ Predictive Analytics
- ประเภทของ ML: Supervised, Unsupervised, Reinforcement
- ML Workflow: จากปัญหาสู่การนำไปใช้

#### 2. Classification
- **Algorithms**: Logistic Regression, Decision Trees, Random Forest
- **Metrics**: Accuracy, Precision, Recall, F1-score
- **Applications**: Loan default prediction

#### 3. Regression
- **Algorithms**: Linear, Polynomial, Ridge, Lasso, Random Forest
- **Metrics**: MAE, MSE, RMSE, R²
- **Applications**: Interest rate prediction

#### 4. Best Practices
- Data quality และ preprocessing
- Model evaluation และ validation
- Deployment considerations
- Ethical AI และ bias detection

### Key Takeaways
1. **เริ่มต้นง่ายๆ**: Simple models often work well
2. **ข้อมูลคือกุญแจ**: Good data > Complex algorithms
3. **Validate เสมอ**: Use cross-validation and holdout sets
4. **Think business**: Focus on solving real problems
5. **Be ethical**: Consider fairness and bias

### การเรียนรู้ต่อไป
- ฝึกฝนกับ datasets ใหม่ๆ
- เรียนรู้ Deep Learning
- ลองทำ end-to-end projects
- เข้าร่วมชุมชน ML

**ขอให้โชคดีในการเดินทาง Machine Learning! 🚀**